In [72]:
# load package
import numpy as np
import pandas as pd
from collections import Counter
import lightgbm as lgb
#from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation  import train_test_split
from sklearn import metrics
from sklearn import cross_validation
import pickle

# set path
import os
default_path = "/Users/francislin/jrml/What's Cooking/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [51]:
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)

In [52]:
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns
del(df)

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_snapper', 'used_graham', 'used_tortellini', 'used_bones',
       'used_methi', 'used_cachaca', 'used_hamburger', 'used_candied',
       'used_grapeseed', 'used_superfine'],
      dtype='object', length=702)

In [53]:
#dataTrain = dataTrain
#去掉y
dataTest = dataTest.drop('cuisine',axis=1)
#去掉key
X_feature = dataTrain.drop(['id','cuisine'],axis=1)
y_t =dataTrain['cuisine']

In [54]:
X_feature.head()

,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,used_onions,used_cheese,...,used_snapper,used_graham,used_tortellini,used_bones,used_methi,used_cachaca,used_hamburger,used_candied,used_grapeseed,used_superfine
0,1,0,0,1,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
#把上面dataTrain.columns的結果貼到dataTrain，再去掉'Survived'
from sklearn.model_selection import train_test_split
#拿掉feature_importances低的因子
X_train, X_test, y_train, y_test = train_test_split(X_feature, y_t, test_size=0.3, random_state=13)

In [13]:
# Function for Measure Performance
from  sklearn  import  metrics
def measure_performance(X,y,clf, show_accuracy=True, show_classification_report=True, show_confusion_matrix=True, show_roc_auc = True):
    y_pred = clf.predict(X)
    y_predprob = clf.predict_proba(X)[:,1]
    if show_accuracy:
        print ("Accuracy:{0:.3f}".format(metrics.accuracy_score(y,y_pred))),"\n"

    if show_classification_report:
        print("Classification report")
        print(metrics.classification_report(y,y_pred)),"\n"
        
    if show_confusion_matrix:
        print("Confusion matrix")
        print(metrics.confusion_matrix(y,y_pred)),"\n"  
        
    if show_roc_auc:
        print("ROC AUC Score")
        print(metrics.roc_auc_score(y,y_predprob)),"\n"

In [ ]:
X_train_array = X_train.values
X_test_array = X_test.values

In [92]:
%%time
import lightgbm as lgb
lgb = lgb.LGBMClassifier(object='multiclass', num_class=20, max_depth=8,learning_rate=0.25)
lgb_fit=lgb.fit(X_train,y_train)

CPU times: user 54.2 s, sys: 11.1 s, total: 1min 5s
Wall time: 26.7 s


In [93]:
y_pred= lgb_fit.predict(X_test)

In [94]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.66079295, 0.44327177, 0.75581395, 0.79023823, 0.65306122,
       0.61969993, 0.75808937, 0.87130802, 0.52631579, 0.84573218,
       0.75352113, 0.74805195, 0.78372591, 0.90858161, 0.75      ,
       0.44052863, 0.74167873, 0.5607064 , 0.76278119, 0.59016393])

0.6982031453786768

0.7814464091175731

In [88]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[  72,    1,    2,    1,    3,    4,    0,    0,    0,   20,    0,
           1,    0,   13,    0,    0,   11,    1,    1,    1],
       [   0,   68,    2,    1,    0,   25,    1,    4,   17,   31,    3,
           0,    0,    3,    0,    4,   60,    0,    0,    0],
       [   0,    2,  309,    1,    3,   17,    1,    1,    0,   28,    1,
           0,    0,   20,    1,    0,   70,    2,    0,    0],
       [   1,    0,    2,  672,    6,    9,    2,    3,    0,   11,    0,
          22,   21,   10,    1,    0,   32,    0,   22,   14],
       [  10,    0,    0,   34,  130,    3,    0,    2,    0,   13,    0,
           3,    1,    6,    0,    0,   29,    1,    9,    8],
       [   1,    7,    7,    2,    3,  437,    7,    2,   11,  176,    3,
           1,    0,    6,    3,    9,   68,   11,    1,    1],
       [   0,    1,    0,    0,    0,    5,  240,    7,    0,   71,    0,
           0,    1,    6,    4,    0,    6,    5,    0,    0],
       [   1,    1,    0,    4,    0,    

In [42]:
# Grid Search
print('Start training...')
estimator = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0) 

param_grid = {
    'learning_rate': [0.11], 
    'num_leaves': [32,64],
    'feature_fraction': [0.3,0.8],
    'bagging_fraction': [0.3,0.9],
    'max_depth':[9],
    #'max_bin': [6],
    'bagging_freq':[4],
    'lambda_l1': [0.1,3,20] , #[44,133] then 44 [44,88] then 44 np.arange(38,44,1) np.arange(16,20,1)
    'lambda_l2': [0.1,1,7] ,#np.arange(11,15,1) then 13
    #'min_split_gain': [0.04], 
    'min_child_weight': [2] 
}

LGBM_grid = GridSearchCV(estimator, param_grid,scoring='accuracy')#scoring='f1_macro'
LGBM_grid.fit(X_train, y_train)
print('Best parameters found by grid search are:', LGBM_grid.best_params_)

# Final Model
evals_result = {} 
print('Start predicting...')
LGBM_grid_final = lgb.LGBMClassifier(boosting_type='gbdt', object='multiclass', num_class=20, random_state=0,
                                     learning_rate = LGBM_grid.best_params_['learning_rate'],
                              num_leaves = LGBM_grid.best_params_['num_leaves'],
                                feature_fraction = LGBM_grid.best_params_['feature_fraction'], 
                             bagging_fraction = LGBM_grid.best_params_['bagging_fraction'],
                             max_depth = LGBM_grid.best_params_['max_depth'],
                              #max_bin = LGBM_grid.best_params_['max_bin'],
                                bagging_freq = LGBM_grid.best_params_['bagging_freq'],
                              lambda_l1 = LGBM_grid.best_params_['lambda_l1'],
                              lambda_l2 = LGBM_grid.best_params_['lambda_l2'],
                            #min_split_gain = LGBM_grid.best_params_['min_split_gain'],
                            min_child_weight = LGBM_grid.best_params_['min_child_weight']
                                    )
LGBM_grid_final_fit = LGBM_grid_final.fit(X_train, y_train)
print("Predicting is over")

Start training...


KeyboardInterrupt: 

In [ ]:
#對test算performance
LGBM_grid_measure = measure_performance(X = X_test, y = y_test, clf = LGBM_grid_final, show_classification_report=True, show_confusion_matrix=True)

In [33]:
#拿全部的dataTrain來建
LGBM_grid_final_fit_all = LGBM_grid_final.fit(X_feature, y_t)

In [34]:
LGBM_grid_final_fit_res =LGBM_grid_final_fit_all.predict(dataTest)

In [18]:
dataTest = dataTest.drop('id',axis=1)

In [35]:
LGBM_grid_final_fit_res

array(['irish', 'southern_us', 'italian', ..., 'italian', 'southern_us',
       'mexican'], dtype=object)

In [36]:
submit=pd.read_csv('./data/sample_submission.csv')

In [37]:
submit['cuisine'] = LGBM_grid_final_fit_res

In [38]:
%pwd
submit.to_csv('submit_to_kaggle.csv', index= False)

"/Users/francislin/JrML/What's Cooking"

In [39]:
submit.cuisine[0]

'irish'

In [56]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500,max_features=10)
rf_clf.fit(X_train, y_train)

CPU times: user 39.2 s, sys: 1.92 s, total: 41.2 s
Wall time: 41.3 s


In [57]:
y_pred= rf_clf.predict(X_test)

In [58]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.57425743, 0.3006993 , 0.72527473, 0.78528281, 0.57519789,
       0.56752874, 0.6618705 , 0.86135094, 0.45936396, 0.80072202,
       0.66135458, 0.71870794, 0.74720358, 0.88161457, 0.69873418,
       0.34408602, 0.70542636, 0.40437158, 0.76416066, 0.54038997])

0.6388798877122986

0.750272353976368

In [65]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
GBDT = GradientBoostingClassifier(n_estimators=100,max_depth=5,learning_rate=0.1)#深度從3變5沒有比較好＠＠
GBDT.fit(X_train,y_train)

CPU times: user 23min 18s, sys: 5.77 s, total: 23min 24s
Wall time: 23min 35s


In [66]:
y_pred= GBDT.predict(X_test)

In [67]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.56896552, 0.36144578, 0.72320377, 0.79151943, 0.56351039,
       0.58974359, 0.71513353, 0.8681672 , 0.49838188, 0.81472957,
       0.70383275, 0.73533246, 0.75268817, 0.8998467 , 0.74429224,
       0.46082949, 0.70700637, 0.51467269, 0.75409836, 0.61261261])

0.6690006261934475

0.7596580910081288

In [68]:
%%time
from xgboost import XGBClassifier
Xgb = XGBClassifier()
Xgb.fit(X_train,y_train)

CPU times: user 14min 9s, sys: 1.66 s, total: 14min 10s
Wall time: 14min 13s


In [69]:
y_pred= Xgb.predict(X_test)

In [70]:
from sklearn.metrics import f1_score
f1_score(y_test,y_pred,average=None)
f1_score(y_test,y_pred,average='macro')
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

array([0.53023256, 0.28571429, 0.71601942, 0.78099652, 0.54881266,
       0.533241  , 0.69952305, 0.85237069, 0.41811847, 0.77361919,
       0.69117647, 0.72112676, 0.69757174, 0.87182096, 0.71904762,
       0.3814433 , 0.63846928, 0.47355164, 0.74428274, 0.58064516])

0.6328891759743992

0.7278136260789407